## list value update for numerical sequence without changing index


In [1]:
import numpy as np

In [54]:
labels = [1, 2, 1, 4, 0, 6]

In [55]:
unique_seq, unique_index = np.unique(labels, return_index=True)
for seq, index in zip(unique_seq, unique_index):
    print('{}--{}'.format(index, seq))

4--0
0--1
1--2
3--4
5--6


In [56]:
sort = sorted(unique_index)
new = [labels[i] for i in sort]
pairs = {v : k for k,v in enumerate(new)}
pairs

{1: 0, 2: 1, 4: 2, 0: 3, 6: 4}

In [57]:
new_label = []
for v in labels:
    new_label.append(pairs[v])
print(new_label)

[0, 1, 0, 2, 3, 4]


In [58]:
def sort_speakers(labels):
    _, index = np.unique(labels, return_index=True)
    pairs = {labels[v]:k for k,v in enumerate(sorted(index))}
    sorted_labels = [pairs[label] for label in labels]
    return sorted_labels

In [59]:
new_labels = sort_speakers(labels)
print(f"Old labels\n{labels}\n")
print(f"New labels\n{new_labels}")

Old labels
[1, 2, 1, 4, 0, 6]

New labels
[0, 1, 0, 2, 3, 4]


## trimming audio with duration(bytes data)

In [63]:
from tools import wavSplit
import os
audio_path = r'/home/zmh/hdd/Custom_Projects/Speaker-Diarization/test-data'
wavfile = "Mark Zuckerberg's 2004 Interview See How Far He And Facebook Have Come - YouTube.wav"


In [69]:
audio, sample_rate, audio_length = wavSplit.read_wave(os.path.join(audio_path, wavfile))
print(len(audio),',' ,audio_length, 'minutes')

224.2699375 , 224.2699375 minutes


In [78]:
audio[-20:]

b'\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'

3.7378322916666664